# Forecasting Weather using Auto Regression and Moving Average model (ARIMA)

## PRANAV THIAGARAJAN UMAPATHY 
## Student ID- 220366757

### Importing required packages

In [ ]:
import math
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import scipy.io
import seaborn as sns
sns.set_style("darkgrid")
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings('ignore')
import statsmodels.tsa.arima.model
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
#Reading the CSV file into a Pandas DataFrame
weather_data = pd.read_csv("weather_dt.csv")

#Printing the DataFrame
print(weather_data.head())

In [ ]:
#Creating new df with windspeed and datetime columns
wind_df = weather_data[['datetime', 'windspeed']].copy()

#printing the first few rows of the DataFrame
print(wind_df.head())

In [ ]:
#formatting datetime column
wind_df["datetime"] = pd.to_datetime(wind_df["datetime"], format='%Y-%m-%d')

In [ ]:
#Filter the data for required time period
start_date = pd.to_datetime("2021-10-01")
end_date = pd.to_datetime("2023-09-30")
filtered_data = wind_df[(wind_df["datetime"] >= start_date) & (wind_df["datetime"] <= end_date)]

In [ ]:
#Convert the 'datetime' column to datetime format
filtered_data['datetime'] = pd.to_datetime(filtered_data['datetime'])

In [ ]:
#Calculate rolling mean and standard deviation
rolling_mean = filtered_data['windspeed'].rolling(window=30).mean()  # Adjust window size as needed
rolling_std = filtered_data['windspeed'].rolling(window=30).std()    # Adjust window size as needed

#Plot the original time series, rolling mean, and rolling standard deviation
plt.figure(figsize=(12, 6))
plt.plot(filtered_data['windspeed'], label='Original Time Series')
plt.plot(rolling_mean, label='Rolling Mean', color='red')
plt.plot(rolling_std, label='Rolling Std Dev', color='green')

plt.title('Time Series with Rolling Mean and Standard Deviation')
plt.xlabel('Datetime')
plt.ylabel('Windspeed')
plt.legend()
plt.show()

In [ ]:
#Plotting a time series plot
time_series = filtered_data.set_index('datetime')['windspeed']

#Plot the original time series and the forecast
plt.figure(figsize=(12, 6))
plt.plot(time_series, label='Original Time Series')
# plt.title('Time Series Plot')
plt.xlabel('Datetime')
plt.ylabel('Windspeed')
plt.legend()
plt.show()

In [ ]:
#Assume 'windspeed' is the target variable
X = filtered_data[['datetime']]
y = filtered_data['windspeed']

#Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

#Display the shapes of the resulting sets
print(f"Train set shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test set shape: X_test={X_test.shape}, y_test={y_test.shape}")

#Sort the training and testing sets based on 'datetime'
X_train = X_train.sort_values(by='datetime')
y_train = y_train.loc[X_train.index]

X_test = X_test.sort_values(by='datetime')
y_test = y_test.loc[X_test.index]

In [ ]:
#Plotting the training set
plt.figure(figsize=(12, 6))
plt.plot(X_train, y_train, label='Training Set', color='blue')
# plt.title('Time Series Plot - Training Set')
plt.xlabel('Datetime')
plt.ylabel('Windspeed')
plt.xticks(rotation=50)
plt.legend()
plt.show()

#Plotting the testing set
plt.figure(figsize=(12, 6))
plt.plot(X_test, y_test, label='Testing Set', color='green')
# plt.title('Time Series Plot - Testing Set')
plt.xlabel('Datetime')
plt.ylabel('Windspeed')
plt.xticks(rotation=50) 
plt.legend()
plt.show()

In [ ]:
#Plot ACF
plt.figure(figsize=(12, 6))
plot_acf(y_train, lags=40, ax=plt.gca())
plt.show()

#Plot PACF
plt.figure(figsize=(12, 6))
plot_pacf(y_train, lags=40, ax=plt.gca())
plt.show()

In [ ]:
#Performing the ADF test on the training set
result = adfuller(y_train)

# Extract and print the results
adf_statistic = result[0]
p_value = result[1]
critical_values = result[4]

print(f'ADF Statistic: {adf_statistic}')
print(f'p-value: {p_value}')
print('Critical Values:')
for key, value in critical_values.items():
    print(f'   {key}: {value}')

In [ ]:
#Fitting an ARIMA model
order = (2, 0, 2)
model = ARIMA(y_train, order=order)
fit_model = model.fit()

#Display model summary
print(fit_model.summary())

In [ ]:
#Fit the ARIMA(2, 0, 2) model on the training set
order = (2, 0, 2)
model = ARIMA(y_train, order=order)
fit_model = model.fit()

#Plot the residuals diagnostics
fit_model.plot_diagnostics(figsize=(12, 8))
plt.show()

In [ ]:
residuals = fit_model.resid
plt.figure(figsize=(12, 6))
plt.plot(residuals)
# plt.title('Residuals Plot')
plt.xlabel('Time')
plt.ylabel('Residuals')
plt.show()

In [ ]:
#Get residuals from the fitted model
residuals = fit_model.resid

#Plot the density of residuals
plt.figure(figsize=(12, 6))
sns.histplot(residuals, kde=True, color='blue')
plt.title('Density Plot of Residuals')
plt.xlabel('Residuals')
plt.ylabel('Density')
plt.show()

In [ ]:
#Fit the ARIMA(2, 0, 2) model on the training set
order = (2, 0, 2)
model = ARIMA(y_train, order=order)
fit_model = model.fit()

#Get the residuals
residuals = fit_model.resid

#Plot ACF of residuals
plt.figure(figsize=(12, 6))
plot_acf(residuals, lags=50, ax=plt.gca())
plt.title('Autocorrelation Function (ACF) of Residuals')
plt.show()

#Plot PACF of residuals
plt.figure(figsize=(12, 6))
plot_pacf(residuals, lags=50, ax=plt.gca())
plt.title('Partial Autocorrelation Function (PACF) of Residuals')
plt.show()

### Forecasting

In [ ]:
#Fit the ARIMA(2, 0, 2) model on the training set
order = (2, 0, 0)
model = ARIMA(y_train, order=order)
fit_model = model.fit()

#Get the forecast for the test set
forecast_steps = len(y_test)
forecast = fit_model.get_forecast(steps=forecast_steps)

#Extract the predicted values and confidence intervals
predicted_values = forecast.predicted_mean
confidence_intervals = forecast.conf_int()

#Plot the actual vs predicted values
plt.figure(figsize=(12, 6))
plt.plot(X_train, y_train, label='Training Data')
plt.plot(X_test, y_test, label='Actual Test Data', color='orange')
plt.plot(X_test, predicted_values, label='ARIMA(2, 0, 2) Predictions(Mean)', color='green')

#Fill the area between the confidence intervals
plt.fill_between(X_test['datetime'], confidence_intervals.iloc[:, 0], confidence_intervals.iloc[:, 1], color='green', alpha=0.2)

plt.title('ARIMA Model - Actual vs Predicted')
plt.xlabel('Datetime')
plt.ylabel('Windspeed')
plt.legend()
plt.show()

### 1-step ahead forecast

In [ ]:
#Perform 1-step ahead forecast
forecast_1_step = fit_model.get_forecast(steps=1)
predicted_values_1_step = forecast_1_step.predicted_mean
confidence_intervals_1_step = forecast_1_step.conf_int()

#Plot the actual vs predicted values with 1-step ahead forecast
plt.figure(figsize=(12, 6))
plt.plot(X_test, y_test, label='Actual Test Data', color='orange')
plt.plot(X_test, predicted_values, label='ARIMA(2, 0, 2) Predictions', color='green')

#1-step ahead forecast
plt.plot(X_test.iloc[0], predicted_values_1_step.iloc[0], marker='o', markersize=8, color='red', label='1-Step Ahead Forecast')

#Fill the area between the confidence intervals for 1-step ahead forecast
plt.fill_between(X_test.iloc[0], confidence_intervals_1_step.iloc[0, 0], confidence_intervals_1_step.iloc[0, 1], color='red', alpha=0.2)

plt.title('ARIMA Model - Actual vs Predicted with 1-step ahead forecast')
plt.xlabel('Datetime')
plt.ylabel('Windspeed')
plt.legend()
plt.show()

### 2-step ahead forecast

In [ ]:
#Perform 2-step ahead forecast
forecast_2_step = fit_model.get_forecast(steps=2)
predicted_values_2_step = forecast_2_step.predicted_mean
confidence_intervals_2_step = forecast_2_step.conf_int()

#Plot the actual vs predicted values with 2-step ahead forecast
plt.figure(figsize=(12, 6))
# plt.plot(X_train, y_train, label='Training Data')
plt.plot(X_test, y_test, label='Actual Test Data', color='orange')
plt.plot(X_test, predicted_values, label='ARIMA(2, 0, 2) Predictions', color='green')

#2-step ahead forecast
plt.plot(X_test.iloc[:2], predicted_values_2_step, marker='o', markersize=8, color='blue', label='2-Step Ahead Forecast')

#Fill the area between the confidence intervals for 2-step ahead forecast
plt.fill_between(X_test.iloc[:2]['datetime'], confidence_intervals_2_step.iloc[:, 0], confidence_intervals_2_step.iloc[:, 1], color='blue', alpha=0.2)

plt.title('ARIMA Model - Actual vs Predicted with 2-step ahead forecasts')
plt.xlabel('Datetime')
plt.ylabel('Windspeed')
plt.legend()
plt.show()

### Model Evaluation

In [ ]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Calculate MSE
mse = mean_squared_error(y_test, predicted_values)
print(f'Mean Squared Error (MSE): {mse}')

# Calculate MAE
mae = mean_absolute_error(y_test, predicted_values)
print(f'Mean Absolute Error (MAE): {mae}')